In [24]:
import numpy as np
import pandas as pd

predict_data = pd.read_csv('../data/processed/val_data.csv')

predict_data

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,0.0,0.259785,0.300643,0.257757,0.143542,0.424483,0.265076,0.187559,0.189911,0.436869,...,0.250445,0.336354,0.227302,0.121092,0.471703,0.204238,0.213339,0.348797,0.285630,0.212908
1,1.0,0.565999,0.392289,0.551517,0.418452,0.338178,0.256181,0.253046,0.395179,0.263636,...,0.602277,0.388060,0.575178,0.413095,0.317837,0.200163,0.214617,0.614777,0.194362,0.071166
2,1.0,0.401297,0.330402,0.400180,0.256797,0.510698,0.315686,0.343486,0.401938,0.439899,...,0.403059,0.372601,0.370985,0.238596,0.550948,0.205790,0.302796,0.520275,0.250739,0.164961
3,0.0,0.256472,0.269530,0.260383,0.137561,0.476393,0.344212,0.181373,0.139115,0.379293,...,0.176094,0.290245,0.195229,0.081252,0.487552,0.228590,0.191933,0.253265,0.195348,0.252853
4,0.0,0.215770,0.159959,0.213254,0.110032,0.426198,0.284093,0.157849,0.128926,0.382828,...,0.154038,0.204158,0.141292,0.066998,0.418213,0.179013,0.143530,0.237732,0.150601,0.172504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,0.0,0.362488,0.241461,0.348421,0.221633,0.304956,0.146003,0.121649,0.138718,0.176263,...,0.303451,0.357676,0.276856,0.158720,0.284158,0.174356,0.194649,0.269003,0.175439,0.071625
110,1.0,0.430167,0.336152,0.416765,0.285981,0.352532,0.198945,0.228889,0.329920,0.372727,...,0.421202,0.334222,0.390408,0.255063,0.520571,0.171930,0.225958,0.522337,0.213877,0.057917
111,0.0,0.214823,0.176530,0.207864,0.111474,0.439379,0.180050,0.101406,0.145577,0.415657,...,0.167912,0.244403,0.151751,0.075354,0.447269,0.127010,0.144089,0.330172,0.216637,0.151187
112,0.0,0.342610,0.613460,0.336950,0.203775,0.267220,0.259248,0.258435,0.219085,0.142424,...,0.278193,0.760128,0.259425,0.142253,0.243875,0.208895,0.244728,0.282440,0.064065,0.150335


### Initialization

In [25]:
def init(data, hidden_layer_nb=2, outputs_nb=2, weights_initializer='heUniform', hidden_nodes_nb=None):
    y_train = data.iloc[:, 0]
    X_train = data.iloc[:, 1:]
    
    if hidden_nodes_nb is None:
        hidden_nodes_nb = int(((X_train.shape[1] + outputs_nb) / 2))

    weights = []
    biases = []
    
    for layer in range(hidden_layer_nb + 1):
        # Input Layer to Hidden Layer
        if layer == 0:
            nodes_in = X_train.shape[1]
            nodes_out = hidden_nodes_nb
        # Hidden Layer to Hidden Layer
        elif layer < hidden_layer_nb:
            nodes_in = hidden_nodes_nb
            nodes_out = hidden_nodes_nb
        # Hidden Layer to Output Layer
        else:
            nodes_in = hidden_nodes_nb
            nodes_out = outputs_nb
            
        if weights_initializer == 'xavier':
            limit = np.sqrt(6 / (nodes_in + nodes_out))
        else:
            limit = np.sqrt(6 / nodes_in)
        weights.append(np.random.uniform(-limit, limit, (nodes_out, nodes_in)))
        biases.append(np.zeros(nodes_out))
    
    return weights, biases, X_train, y_train
    
weights, biases, X_train, y_train = init(predict_data)

for i in range(len(weights)):
    print(f'weights[{i}]: {weights[i].shape}')
    print(f'biases[{i}]: {biases[i].shape}\n')



weights[0]: (16, 30)
biases[0]: (16,)

weights[1]: (16, 16)
biases[1]: (16,)

weights[2]: (2, 16)
biases[2]: (2,)



### Activation Functions

In [26]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    exps = np.exp(x - x.max())
    return exps / np.sum(exps, axis=1, keepdims=True)

### Forward Propagation

In [27]:
def forward_propagation(X, weights, biases, activation='sigmoid', output_activation='softmax'):
    layers = [X]
    for i in range(len(weights)):
        if i == len(weights) - 1:
            if output_activation == 'softmax':
                layers.append(pd.DataFrame(softmax(np.dot(layers[i], weights[i].T) + biases[i])))
        else:
            if activation == 'sigmoid':
                layers.append(pd.DataFrame(sigmoid(np.dot(layers[i], weights[i].T) + biases[i])))
    return layers

layers = forward_propagation(X_train, weights, biases)
layers[3]

,0,1
0,0.383021,0.616979
1,0.384272,0.615728
2,0.379009,0.620991
3,0.383337,0.616663
4,0.384650,0.615350
...,...,...
109,0.387037,0.612963
110,0.382109,0.617891
111,0.384211,0.615789
112,0.386680,0.613320


In [28]:
for i in range(len(layers)):
    print(f'layers[{i}]: {layers[i].shape}')

layers[0]: (114, 30)
layers[1]: (114, 16)
layers[2]: (114, 16)
layers[3]: (114, 2)


### Get Predicted Classes to 'M' or 'B'

In [33]:
y_pred = layers[3].idxmax(axis=1)

# y_pred = y_pred.apply(lambda x: 'M' if x == 1 else 'B')
# df = pd.DataFrame(y_pred)
# 
# df.to_csv('../data/trained/trained.csv', index=False)
# df

### Evaluate Model

In [35]:
def evaluate(y_train, y_pred, loss='binary_cross_entropy'):
    if loss == 'binary_cross_entropy':
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        N = len(y_train)

        loss = -(1/N) * np.sum(
            y_train * np.log(y_pred) + 
            (1 - y_train) * np.log(1 - y_pred)
        )
    return loss
# print(y_train, y_pred
#       )
evaluate(y_train, y_pred)



/Users/quentin/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


inf